[Code reference](https://medium.com/@sebastingarcaacosta/how-to-export-a-tensorflow-2-x-keras-model-to-a-frozen-and-optimized-graph-39740846d9eb)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
print(f"TF version: {tf.__version__}")
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np

TF version: 2.8.2


In [3]:
from keras.models import load_model

MODEL_DIR = "/content/drive/MyDrive/Uni/CV Project/Alberto/models/saved_models/simple_model"

# path of the directory where you want to save your model
frozen_out_path = MODEL_DIR

# name of the .pb file
frozen_graph_filename = "frozen_unet"

model = load_model(MODEL_DIR)

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 384, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 390, 230, 3)  0           ['input_13[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 192, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

Convert the Keras model to ConcreteFunction format, which is more general

In [4]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

Once we have our model in the format of ConcreteFunction, we convert its variables to constants

In [5]:
# Get frozen graph def
frozen_func = convert_variables_to_constants_v2(full_model)
# frozen_func.graph.as_graph_def()

We are almost done. If you want to inspect the layers operations inside your frozen graph definition and see the name of its input and output tensors (important for the next stage), use this code



In [6]:
# layers = [op.name for op in frozen_func.graph.get_operations()]
# print("-" * 60)
# print("Frozen model layers: ")
# for layer in layers:
#     print(layer)
# print("-" * 60)
# print("Frozen model inputs: ")
# print(frozen_func.inputs)
# print("Frozen model outputs: ")
# print(frozen_func.outputs)

Then, serialize the frozen graph and its text representation to disk

In [7]:
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pb",
                  as_text=False)
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pbtxt",
                  as_text=True)

'/content/drive/MyDrive/Uni/CV Project/Alberto/models/saved_models/simple_model/frozen_unet.pbtxt'

In [8]:
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.13.1

Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220719082949
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 345.0 MB 3.9 kB/s 
     |████████████████████████████████| 3.2 MB 52.8 MB/s 
     |████████████████████████████████| 367 kB 68.4 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0


In this stage, we’ll use a helper function in order to optimize the graph for inference available in TensorFlow 1.x

In [1]:
import tensorflow as tf
print(f"TF version: {tf.__version__}")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

TF version: 1.13.1


In [2]:
!python -m tensorflow.python.tools.optimize_for_inference --input "/content/drive/MyDrive/Uni/CV Project/Alberto/models/saved_models/simple_model/frozen_unet.pb" \
--output "/content/drive/MyDrive/Uni/CV Project/Alberto/models/saved_models/simple_model/optimized_unet.pb" --frozen_graph=True --input_names=x --output_names=Identity

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [11]:
# Needs tensorflow 1.5
import tensorflow as tf

MODEL_DIR = "/content/drive/MyDrive/Uni/CV Project/Alberto/models/saved_models/simple_model"

optimized_graph_path = MODEL_DIR + "/optimized_unet.pb"
output_pbtxt = MODEL_DIR + "/optimized_unet.pbtxt"

# Read the graph.
with tf.gfile.FastGFile(optimized_graph_path, "rb") as f:
  graph_def = tf.GraphDef()
  graph_def.ParseFromString(f.read())

# Remove Const nodes.
for i in reversed(range(len(graph_def.node))):
  if graph_def.node[i].op == 'Const' or graph_def.node[i].op == 'Mul' or graph_def.node[i].op == 'ResizeBilinear' \
  or graph_def.node[i].op == 'ConcatV2' or graph_def.node[i].op == 'Conv2D' or graph_def.node[i].op == 'FusedBatchNormV3':
      del graph_def.node[i]
  print(graph_def.node[i].op)
  for attr in ['T', 'data_format', 'Tshape', 'N', 'Tidx', 'Tdim',
                'use_cudnn_on_gpu', 'Index', 'Tperm', 'is_training',
                'Tpaddings']:
      if attr in graph_def.node[i].attr:
          del graph_def.node[i].attr[attr]

# Save as text.
tf.train.write_graph(graph_def, "", output_pbtxt, as_text=True)

FusedResizeAndPadConv2D
FusedResizeAndPadConv2D
Identity
NoOp
BiasAdd
BiasAdd
BiasAdd
BiasAdd
BiasAdd
BiasAdd
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
BatchToSpaceND
BatchToSpaceND
BatchToSpaceND
BatchToSpaceND
BatchToSpaceND
SpaceToBatchND
SpaceToBatchND
SpaceToBatchND
Relu
Relu
Relu
Relu
Relu
Relu
BatchToSpaceND
BatchToSpaceND
BatchToSpaceND
BatchToSpaceND
BatchToSpaceND
SpaceToBatchND
SpaceToBatchND
SpaceToBatchND
Relu
Relu
Relu
Relu
Relu
Relu
BatchToSpaceND
BatchToSpaceND
BatchToSpaceND
BatchToSpaceND
BatchToSpaceND
SpaceToBatchND
SpaceToBatchND
SpaceToBatchND
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
Relu
BiasAdd
BiasAdd
BiasAdd
BiasAdd
AvgPool
Relu
Relu
Relu
Relu
Relu
Relu
BiasAdd
BiasAdd
BiasAdd
BiasAdd
Relu
Relu
Relu
Relu
Relu
Relu
BiasAdd
BiasAdd
Bi

'/content/drive/MyDrive/Uni/CV Project/Alberto/models/saved_models/simple_model/optimized_unet.pbtxt'